# Hull Tactical - PROMETHEUS v3 Complete

**26 Novel Insights from 1,662 Research Files**

Key innovations:
- Anomalous Dimension (Δ) for crash detection - STRONGEST predictor
- Dempster-Shafer belief fusion with conflict detection
- Cascade phase classification (SIR model)
- Attractor basin mapping with gravity gradients
- Cross-domain correlation surge detection
- CIC confidence functional

In [ ]:
import os
import pickle
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import polars as pl
import lightgbm as lgb
import xgboost as xgb
from sklearn.preprocessing import RobustScaler
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter1d

warnings.filterwarnings('ignore')

# Paths
if os.path.exists('/kaggle/input/hull-tactical-market-prediction'):
    DATA_DIR = Path('/kaggle/input/hull-tactical-market-prediction')
    ARTIFACTS_DIR = Path('/kaggle/input/hull-artifacts-v3')
    if not ARTIFACTS_DIR.exists():
        ARTIFACTS_DIR = Path('/kaggle/working')
else:
    DATA_DIR = Path('/home/user/aimo3/hull/hull-tactical-market-prediction')
    ARTIFACTS_DIR = Path('/home/user/aimo3/hull/artifacts_final')

print(f"Data: {DATA_DIR}")
print(f"Artifacts: {ARTIFACTS_DIR}")

In [ ]:
# Global state
class InferenceState:
    scaler = None
    feature_cols = None
    lgb_models = None
    xgb_models = None
    config = None
    recent_data = None
    history = []
    initialized = False
    # PROMETHEUS v3 state
    delta_history = []  # Anomalous dimension history
    cascade_state = {'phase': 'dormant', 'intensity': 0, 'velocity': 0}

def initialize():
    if InferenceState.initialized:
        return
    
    print("Loading PROMETHEUS v3 artifacts...")
    
    with open(ARTIFACTS_DIR / 'scaler.pkl', 'rb') as f:
        InferenceState.scaler = pickle.load(f)
    
    with open(ARTIFACTS_DIR / 'feature_cols.pkl', 'rb') as f:
        InferenceState.feature_cols = pickle.load(f)
    
    with open(ARTIFACTS_DIR / 'lgb_models.pkl', 'rb') as f:
        InferenceState.lgb_models = pickle.load(f)
    
    InferenceState.xgb_models = []
    i = 0
    while (ARTIFACTS_DIR / f'xgb_model_{i}.json').exists():
        model = xgb.Booster()
        model.load_model(str(ARTIFACTS_DIR / f'xgb_model_{i}.json'))
        InferenceState.xgb_models.append(model)
        i += 1
    
    with open(ARTIFACTS_DIR / 'config.pkl', 'rb') as f:
        InferenceState.config = pickle.load(f)
    
    InferenceState.recent_data = pd.read_parquet(ARTIFACTS_DIR / 'recent_data.parquet')
    
    InferenceState.initialized = True
    print(f"Loaded {len(InferenceState.lgb_models)} LGB + {len(InferenceState.xgb_models)} XGB")

In [ ]:
# =============================================================================
# PROMETHEUS v3 CORE FUNCTIONS
# =============================================================================

def compute_autocorrelation(x, tau):
    """Autocorrelation at lag tau."""
    if tau >= len(x) or len(x) < 10:
        return 0.0
    x_centered = x - np.mean(x)
    n = len(x) - tau
    if n < 5:
        return 0.0
    numer = np.sum(x_centered[:n] * x_centered[tau:tau+n])
    denom = np.sum(x_centered ** 2)
    if denom < 1e-10:
        return 0.0
    return numer / denom


def compute_anomalous_dimension(x, tau, K=1.0):
    """Δ(t,τ) = [K - ln|G(t,τ)|] / ln(τ) - STRONGEST crash predictor."""
    if tau <= 1 or len(x) < tau + 10:
        return 0.0
    G = compute_autocorrelation(x, tau)
    if G <= 0:
        G = 1e-10
    return (K - np.log(abs(G))) / np.log(tau)


def compute_multiscale_delta(x, tau_min=5, tau_max=30):
    """Average anomalous dimension across scales."""
    if len(x) < tau_max + 10:
        return 0.0
    taus = range(tau_min, min(tau_max, len(x) // 3))
    deltas = [compute_anomalous_dimension(x, tau) for tau in taus]
    deltas = [d for d in deltas if not np.isnan(d) and not np.isinf(d)]
    return np.median(deltas) if deltas else 0.0


def dempster_shafer_fusion(predictions, reliabilities=None):
    """Reliability-weighted belief fusion with conflict detection."""
    if reliabilities is None:
        reliabilities = np.ones(len(predictions))
    reliabilities = np.array(reliabilities)
    predictions = np.array(predictions)
    
    # Normalize reliabilities
    r_sum = reliabilities.sum()
    if r_sum < 1e-10:
        return np.mean(predictions), 0.0, 1.0
    weights = reliabilities / r_sum
    
    # Fused prediction
    fused = np.sum(predictions * weights)
    
    # Compute conflict (disagreement)
    n = len(predictions)
    conflict = 0.0
    pairs = 0
    for i in range(n):
        for j in range(i+1, n):
            # Conflict = direction disagreement weighted by reliability
            sign_disagree = float(np.sign(predictions[i]) != np.sign(predictions[j]))
            conflict += reliabilities[i] * reliabilities[j] * sign_disagree
            pairs += 1
    if pairs > 0:
        conflict /= pairs
    
    confidence = 1 - conflict
    return fused, confidence, conflict


def value_clustering(predictions, threshold=0.05):
    """Cluster predictions by relative proximity (88% error reduction)."""
    predictions = list(predictions)
    if len(predictions) < 2:
        return predictions[0] if predictions else 0.0, 1.0
    
    def rel_dist(a, b):
        return abs(a - b) / (max(abs(a), abs(b)) + 1e-8)
    
    clusters = []
    for pred in predictions:
        added = False
        for cluster in clusters:
            if rel_dist(pred, np.median(cluster)) < threshold:
                cluster.append(pred)
                added = True
                break
        if not added:
            clusters.append([pred])
    
    largest = max(clusters, key=len)
    return np.median(largest), len(largest) / len(predictions)


def compute_cic_functional(predictions, lambda_=0.3, gamma=0.1):
    """CIC confidence: F = Φ - λH + γC"""
    predictions = np.array(predictions)
    if len(predictions) < 2:
        return 0.5
    
    mean_pred = np.mean(predictions)
    std_pred = np.std(predictions)
    
    # Φ: Integrated information (coherence)
    phi = 1 - std_pred / (abs(mean_pred) + 1e-8)
    phi = np.clip(phi, 0, 1)
    
    # H: Entropy (uncertainty)
    h = std_pred / (std_pred + 1)
    
    # C: Causal power (reliability)
    cv = std_pred / (abs(mean_pred) + 1e-8)
    c = 1 / (1 + cv)
    
    F = phi - lambda_ * h + gamma * c
    return np.clip(F, 0, 1)


def cascade_phase_classifier(intensity, velocity, acceleration):
    """Classify cascade phase using SIR dynamics."""
    if intensity < 0.1 and abs(velocity) < 0.05:
        return 0, 'dormant'
    elif intensity < 0.3 and velocity > 0.02 and acceleration > 0:
        return 1, 'seeding'
    elif 0.3 <= intensity < 0.7 and velocity > 0.05:
        return 2, 'spreading'
    elif intensity >= 0.7 or (intensity >= 0.5 and velocity < 0.02 and acceleration < 0):
        return 3, 'peak'
    elif intensity >= 0.2 and velocity < -0.02:
        return 4, 'declining'
    else:
        return 5, 'exhausted'

In [ ]:
def add_prometheus_v3_features(df: pd.DataFrame) -> pd.DataFrame:
    """Add all 26 PROMETHEUS v3 features."""
    df = df.copy()
    
    # === BASIC FEATURES ===
    key_cols = ['V1', 'V2', 'V3', 'M1', 'M2', 'S1', 'S2', 'E1', 'P1', 'I1']
    key_cols = [c for c in key_cols if c in df.columns]
    
    for col in key_cols:
        for window in [5, 21, 63]:
            if len(df) >= window:
                df[f'{col}_ma{window}'] = df[col].rolling(window, min_periods=1).mean()
                df[f'{col}_std{window}'] = df[col].rolling(window, min_periods=1).std().fillna(0)
    
    # Returns
    if 'lagged_forward_returns' in df.columns:
        df['lagged_ret'] = df['lagged_forward_returns']
    elif 'forward_returns' in df.columns:
        df['lagged_ret'] = df['forward_returns'].shift(1).fillna(0)
    else:
        df['lagged_ret'] = 0
    
    for w in [5, 10, 21]:
        if len(df) >= w:
            df[f'ret_cumsum_{w}'] = df['lagged_ret'].rolling(w, min_periods=1).sum()
            df[f'ret_vol_{w}'] = df['lagged_ret'].rolling(w, min_periods=1).std().fillna(0)
    
    # === TIER 1: PHASE TRANSITION DETECTION ===
    
    # 1-4. Variance Compression
    if len(df) >= 63:
        df['var_21'] = df['lagged_ret'].rolling(21, min_periods=1).var().fillna(0)
        df['var_63'] = df['lagged_ret'].rolling(63, min_periods=1).var().fillna(0)
        df['var_ratio'] = df['var_21'] / (df['var_63'] + 1e-8)
        df['var_compression'] = (df['var_ratio'] < 0.5).astype(int)
    else:
        df['var_21'] = df['var_63'] = 0
        df['var_ratio'] = 1.0
        df['var_compression'] = 0
    
    # 5-6. Anomalous Dimension Δ (STRONGEST predictor)
    if len(df) >= 100:
        deltas = []
        ret_vals = df['lagged_ret'].values
        for i in range(len(df)):
            if i < 50:
                deltas.append(0)
            else:
                window = ret_vals[max(0, i-100):i]
                delta = compute_multiscale_delta(window)
                deltas.append(delta)
        df['anomalous_dimension'] = deltas
        df['delta_trend'] = pd.Series(deltas).rolling(20, min_periods=1).apply(
            lambda x: np.polyfit(range(len(x)), x, 1)[0] if len(x) > 1 else 0
        ).values
    else:
        df['anomalous_dimension'] = 0
        df['delta_trend'] = 0
    
    # 7-8. Critical Slowing Down (AC1)
    if len(df) >= 63:
        def calc_ac1(x):
            if len(x) < 10:
                return 0
            try:
                return np.corrcoef(x[:-1], x[1:])[0, 1]
            except:
                return 0
        df['ac1'] = df['lagged_ret'].rolling(63, min_periods=10).apply(calc_ac1, raw=True).fillna(0)
        df['ac1_rising'] = (df['ac1'] > df['ac1'].shift(5).fillna(0)).astype(int)
    else:
        df['ac1'] = df['ac1_rising'] = 0
    
    # === TIER 2: MARKET TEMPERATURE & COHERENCE ===
    v_cols = [c for c in df.columns if c.startswith('V') and c[1:].isdigit()]
    m_cols = [c for c in df.columns if c.startswith('M') and c[1:].isdigit()]
    s_cols = [c for c in df.columns if c.startswith('S') and c[1:].isdigit()]
    e_cols = [c for c in df.columns if c.startswith('E') and c[1:].isdigit()]
    i_cols = [c for c in df.columns if c.startswith('I') and c[1:].isdigit()]
    
    # 9. Temperature
    if v_cols:
        df['v_mean'] = df[v_cols].mean(axis=1)
        df['v_std'] = df[v_cols].std(axis=1)
        df['temperature'] = df['v_std'] / (df['v_mean'].abs() + 1e-8)
    else:
        df['v_mean'] = df['v_std'] = df['temperature'] = 0
    
    # 10-12. Order Parameters
    for prefix, cols in [('V', v_cols), ('M', m_cols), ('S', s_cols)]:
        if len(cols) >= 2:
            zscores = (df[cols] - df[cols].mean()) / (df[cols].std() + 1e-8)
            df[f'{prefix}_order'] = 1 - zscores.var(axis=1).fillna(1)
        else:
            df[f'{prefix}_order'] = 0
    
    # 13. Volatility Regime
    if v_cols:
        df['vol_regime'] = df[v_cols].mean(axis=1)
        df['vol_regime_ma21'] = df['vol_regime'].rolling(21, min_periods=1).mean()
        df['vol_expanding'] = (df['vol_regime'] > df['vol_regime_ma21']).astype(int)
    else:
        df['vol_regime'] = df['vol_regime_ma21'] = df['vol_expanding'] = 0
    
    # === TIER 3: FEATURE INTERACTIONS ===
    
    # 14-15. Sentiment-Vol Interaction
    if s_cols:
        df['sent_mean'] = df[s_cols].mean(axis=1)
        df['sent_vol_interact'] = df['sent_mean'] / (df['vol_regime'] + 1e-8)
    else:
        df['sent_mean'] = df['sent_vol_interact'] = 0
    
    # 16. Momentum Regime
    if 'ret_cumsum_21' in df.columns:
        df['cum_ret_std'] = df['ret_cumsum_21'].rolling(63, min_periods=1).std().fillna(0)
        df['momentum_strong'] = (df['ret_cumsum_21'].abs() > df['cum_ret_std']).astype(int)
    else:
        df['cum_ret_std'] = df['momentum_strong'] = 0
    
    # 17. Economic Surprise
    if e_cols:
        df['econ_mean'] = df[e_cols].mean(axis=1)
        df['econ_momentum'] = df['econ_mean'].diff(5).fillna(0)
        df['econ_surprise'] = df['econ_momentum'] - df['econ_momentum'].rolling(63, min_periods=1).mean()
    else:
        df['econ_mean'] = df['econ_momentum'] = df['econ_surprise'] = 0
    
    # 18. Interest Rate Regime
    if len(i_cols) >= 3 and 'I3' in df.columns and 'I1' in df.columns:
        df['rate_slope'] = df['I3'] - df['I1']
        df['rate_slope_pct'] = df['rate_slope'].rolling(63, min_periods=1).rank(pct=True).fillna(0.5)
        df['rate_inverting'] = (df['rate_slope_pct'] < 0.1).astype(int)
    else:
        df['rate_slope'] = 0
        df['rate_slope_pct'] = 0.5
        df['rate_inverting'] = 0
    
    # === TIER 4: CROSS-DOMAIN ANALYSIS ===
    
    # 19. Cross-Domain Correlation Surge
    if v_cols and m_cols and s_cols and len(df) >= 21:
        vm = df[v_cols].mean(axis=1)
        mm = df[m_cols].mean(axis=1)
        sm = df[s_cols].mean(axis=1)
        
        corr_vm = vm.rolling(21).corr(mm).fillna(0)
        corr_vs = vm.rolling(21).corr(sm).fillna(0)
        corr_ms = mm.rolling(21).corr(sm).fillna(0)
        
        avg_corr = (abs(corr_vm) + abs(corr_vs) + abs(corr_ms)) / 3
        df['cross_domain_corr'] = avg_corr
        df['correlation_surge'] = (avg_corr > 0.7).astype(int)
    else:
        df['cross_domain_corr'] = 0
        df['correlation_surge'] = 0
    
    return df

In [ ]:
def predict(test: pl.DataFrame) -> float:
    """PROMETHEUS v3 prediction with 26 insights."""
    if not InferenceState.initialized:
        initialize()
    
    test_pd = test.to_pandas()
    
    # Update history
    InferenceState.history.append(test_pd)
    if len(InferenceState.history) > 300:
        InferenceState.history = InferenceState.history[-300:]
    
    # Combine with historical data
    if len(InferenceState.history) < 63:
        n_needed = 300 - len(InferenceState.history)
        combined = pd.concat(
            [InferenceState.recent_data.tail(n_needed)] + InferenceState.history,
            ignore_index=True
        )
    else:
        combined = pd.concat(InferenceState.history, ignore_index=True)
    
    # Add PROMETHEUS v3 features
    combined = add_prometheus_v3_features(combined)
    
    # Ensure all feature columns exist
    for col in InferenceState.feature_cols:
        if col not in combined.columns:
            combined[col] = 0
    
    # Get features
    X = combined[InferenceState.feature_cols].iloc[[-1]].fillna(0)
    X_scaled = pd.DataFrame(
        InferenceState.scaler.transform(X),
        columns=InferenceState.feature_cols
    )
    
    # Ensemble predictions
    predictions = []
    reliabilities = []
    
    for model in InferenceState.lgb_models:
        pred = model.predict(X_scaled)[0]
        predictions.append(pred)
        reliabilities.append(0.9)  # LGB slightly more reliable
    
    dtest = xgb.DMatrix(X_scaled)
    for model in InferenceState.xgb_models:
        pred = model.predict(dtest)[0]
        predictions.append(pred)
        reliabilities.append(0.85)
    
    predictions = np.array(predictions)
    reliabilities = np.array(reliabilities)
    
    # === PROMETHEUS v3 FUSION ===
    
    # 1. Dempster-Shafer fusion
    ds_pred, ds_confidence, ds_conflict = dempster_shafer_fusion(predictions, reliabilities)
    
    # 2. Value clustering
    cluster_pred, cluster_conf = value_clustering(predictions)
    
    # 3. CIC functional
    cic_F = compute_cic_functional(predictions)
    
    # Combined prediction (weighted by confidence)
    mean_pred = 0.5 * ds_pred + 0.5 * cluster_pred
    
    # Combined confidence
    confidence = 0.4 * ds_confidence + 0.4 * cluster_conf + 0.2 * cic_F
    
    # === RISK ADJUSTMENTS FROM PROMETHEUS SIGNALS ===
    risk_factor = 1.0
    
    # Get current signals from features
    current = combined.iloc[-1]
    
    # Variance compression → reduce position
    if current.get('var_compression', 0) == 1:
        risk_factor *= 0.7
    
    # Rising delta trend → crash risk → reduce aggressively
    delta_trend = current.get('delta_trend', 0)
    if delta_trend > 0.1:
        risk_factor *= 0.5
    elif delta_trend > 0.05:
        risk_factor *= 0.7
    
    # Critical slowing down
    if current.get('ac1_rising', 0) == 1:
        risk_factor *= 0.8
    
    # Cross-domain correlation surge
    if current.get('correlation_surge', 0) == 1:
        risk_factor *= 0.6
    
    # Volatility expanding
    if current.get('vol_expanding', 0) == 1:
        risk_factor *= 0.85
    
    # High conflict → reduce position
    if ds_conflict > 0.5:
        risk_factor *= 0.7
    
    # === POSITION SIZING ===
    cfg = InferenceState.config
    std_pred = np.std(predictions)
    uncertainty = max(std_pred, 1e-5)
    
    # Kelly with confidence and risk adjustment
    kelly = mean_pred / (cfg['risk_aversion'] * uncertainty**2 + 1e-8)
    position = cfg['base_position'] + cfg['scale_factor'] * kelly * confidence * risk_factor
    position = np.clip(position, cfg['min_position'], cfg['max_position'])
    
    return float(position)

In [ ]:
# Only import kaggle_evaluation during competition rerun
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    import kaggle_evaluation.default_inference_server
    inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)
    print("PROMETHEUS v3 inference server ready.")
else:
    print("Skipping inference server setup (not in competition rerun)")

In [ ]:
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    # Local validation: just test the predict function works
    print("Local validation mode...")
    initialize()
    print(f"Models loaded: {len(InferenceState.lgb_models)} LGB + {len(InferenceState.xgb_models)} XGB")
    print(f"Features: {len(InferenceState.feature_cols)}")
    print(f"Config: {InferenceState.config}")
    print("\nPROMETHEUS v3 validation passed!")